<a href="https://colab.research.google.com/github/SMohata/ZS-Challenge/blob/master/HackerEarth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import re
import nltk
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
  

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer

from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from nltk.classify.scikitlearn import SklearnClassifier
from textblob import TextBlob


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
url = "https://raw.githubusercontent.com/SMohata/ZS-Challenge/master/dataset/train.csv"
train = pd.read_csv(url, encoding = "ISO-8859-1")

url = "https://raw.githubusercontent.com/SMohata/ZS-Challenge/master/dataset/test.csv"
test = pd.read_csv(url, encoding = "ISO-8859-1")

In [0]:
train = train.replace(np.nan, '', regex=True)
test = test.replace(np.nan, '', regex=True)

In [4]:
df = train.append(test, ignore_index=True)
df.head()

,Date(ET),Host,Index,Link,Patient_Tag,Source,TRANS_CONV_TEXT,Time(ET),Title,Unnamed: 9,time(GMT)
0,6/15/2016,cafepharma.com,NaN,http://cafepharma.com/boards/threads/epstein.5...,0.0,FORUMS,I don't disagree with you in principle. I'm ju...,13:58:00,Epstein,NaN,6/15/2016 23:28
1,5/7/2016,www.patient.co.uk,NaN,http://www.patient.co.uk/forums/discuss/enlarg...,1.0,FORUMS,I am always dizzy I get dizzy standing up so I...,0.820833333,Enlarged Heart.Thread Enlarged Heart,NaN,42498.21667
2,4/14/2016,http://abcnewsradioonline.com/entertainment-news,NaN,http://abcnewsradioonline.com/entertainment-ne...,0.0,BLOG,Axelle/Bauer-Griffin/FilmMagic(NEW YORK) -- Qu...,15:00:38,Queen Latifah Joins American Heart Association...,NaN,4/15/2016 0:30
3,6/18/2016,www.cancer-forums.net,NaN,http://www.cancer-forums.net/viewtopic.php?f=1...,1.0,FORUMS,I am 17 and I have been throwing up for about ...,20:46:00,Bulaemia,NaN,6/19/2016 6:16
4,6/15/2016,www.diyaudio.com,NaN,http://www.diyaudio.com/forums/lounge/292252-d...,0.0,FORUMS,Quote: Originally Posted by Boyan Silyavski Wa...,3:26:00,DIY Silver interconnects and RCAs???,NaN,6/15/2016 12:56


In [0]:
import re
i=0;
for s in df['TRANS_CONV_TEXT'].values:
    if (len(re.findall('<.*?>', s))):
        break;
    i += 1;
stop = set(stopwords.words('english'))
lemma = nltk.wordnet.WordNetLemmatizer()

def cleanhtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned




In [0]:
i=0
str1=' '
final_string=[]
all_positive_words=[]
all_negative_words=[]
s=''
for sent in df['TRANS_CONV_TEXT'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(lemma.lemmatize(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (df['Patient_Tag'].values)[i] == 1: 
                        all_positive_words.append(s) 
                    if(df['Patient_Tag'].values)[i] == 0:
                        all_negative_words.append(s)
                else:
                    continue
            else:
                continue 
                
    str1 = b" ".join(filtered_sentence)
    
    final_string.append(str1)
    i+=1

In [0]:
df['CleanedText']=final_string 
df['CleanedText']=df['CleanedText'].str.decode("utf-8")

In [0]:
tf_idf_vect = TfidfVectorizer(analyzer='word', strip_accents='ascii')
transmess = tf_idf_vect.fit_transform(df['CleanedText'])
tfidf_transformer = TfidfTransformer().fit(transmess)
messages_tfidf = tfidf_transformer.transform(transmess)

In [0]:
train_bow = transmess[:1157,:]
test_bow = transmess[1157:,:]

In [0]:
xtrain, xtest, ytrain, ytest = train_test_split(train_bow, train['Patient_Tag'],test_size=0.2,random_state=1)

In [11]:
svm = (SVC(kernel='sigmoid',probability=True, ))
svm.fit(xtrain,ytrain)
print(classification_report( svm.predict(xtest) , ytest))
print(accuracy_score( svm.predict(xtest) , ytest ))

              precision    recall  f1-score   support

           0       0.97      0.90      0.93       201
           1       0.56      0.81      0.66        31

    accuracy                           0.89       232
   macro avg       0.76      0.85      0.80       232
weighted avg       0.91      0.89      0.90       232

0.8879310344827587


In [12]:
y_pred = svm.predict(xtest)
print("Accuracy on test set: %0.3f%%"%(accuracy_score(ytest, y_pred)*100))
print("Precision on test set: %0.3f"%(precision_score(ytest, y_pred,average='macro')))
print("Recall on test set: %0.3f"%(recall_score(ytest, y_pred,average='macro')))
print("F1-Score on test set: %0.3f"%(f1_score(ytest, y_pred,average='macro')))

Accuracy on test set: 88.793%
Precision on test set: 0.853
Recall on test set: 0.762
F1-Score on test set: 0.795


In [13]:
test_pred = svm.predict_proba(test_bow)
test_pred_int = test_pred[:,1] >= 0.3
test_pred_int = test_pred_int.astype(np.int)

test['Patient_Tag'] = test_pred_int
submission = test[['Index','Patient_Tag']]

submission.head()

,Index,Patient_Tag
0,1,0
1,2,1
2,3,0
3,4,1
4,5,0


In [0]:
from google.colab import drive
drive.mount('drive')

In [0]:
submission.to_csv('submit4.csv', index=False)
#!cp submit4.csv "drive/My Drive/"